<h1 align='center'>Text Generation with LSTM</h1>

In this section we will be using LSTM based deep learning model to generate texts.

I will be using writing of shakespeare as a texts corpous data for traning our model.

Seems exciting, Let's start building it.


## Imports


In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

## Load the data

In [3]:
pwd

'/Users/ajitkumarsingh/Desktop/Hands-on-with-PyTorch/Time Series Analysis with PyTorch/notebooks'

In [13]:
with open('../data/shakespeare.txt','r',encoding='utf8') as f:
    text_data = f.read()

In [14]:
len(text_data)

5445609

So there are around `5.4M` characters in the above data.

Let me display some lines out of the whole data.


In [15]:
print(text_data[:100])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose mi


## Data preparation

Text data being the unstructured one is somewhat complex to deal with. Data preparation is crucial part to feed language models. 

We need to encode the above texts into some numbers. We are building a character level model like the prediction of the model will be a single character at each time stamp. We need to encode it on character level.


In [17]:
#get all distinct characters 

all_char = set(text_data)

print("Total number of distinct characters are : {}".format(len(all_char)))

Total number of distinct characters are : 84


In [39]:
all_char

{'\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '>',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 '}'}

Let's create a encoder and decoder object. Encoder is nothing but a lookup table which return a integer value for each character in `text_data` and decoder is just opposite in the objective like it takes a integer value and return the character corresponding to it.

In [42]:
"\n" in dict(enumerate(all_char))

False

In [38]:
#decoder

decoder = dict(enumerate(all_char))

"\n" in decoder

False

In [33]:
decoder.items()

dict_items([(0, 'H'), (1, 'K'), (2, '}'), (3, 'C'), (4, ';'), (5, '('), (6, 'T'), (7, 'I'), (8, 'j'), (9, 'L'), (10, 'u'), (11, ')'), (12, 'N'), (13, 'V'), (14, 'F'), (15, '\n'), (16, 'r'), (17, '0'), (18, 'B'), (19, 'D'), (20, 'E'), (21, '8'), (22, 'g'), (23, 'm'), (24, 'O'), (25, ','), (26, 'h'), (27, 'A'), (28, 'P'), (29, 'J'), (30, ':'), (31, 'k'), (32, 'c'), (33, '`'), (34, 'S'), (35, '6'), (36, '2'), (37, 'x'), (38, 'v'), (39, 'U'), (40, '"'), (41, '-'), (42, 'i'), (43, 'W'), (44, 'X'), (45, '1'), (46, 's'), (47, '|'), (48, ' '), (49, 'w'), (50, '!'), (51, 'R'), (52, 'Y'), (53, 'p'), (54, '&'), (55, '5'), (56, 'Q'), (57, 'y'), (58, 'n'), (59, '['), (60, '?'), (61, '9'), (62, 'b'), (63, 'a'), (64, 'z'), (65, '3'), (66, 'e'), (67, '.'), (68, 'M'), (69, '7'), (70, 'G'), (71, 't'), (72, 'Z'), (73, 'd'), (74, "'"), (75, '<'), (76, 'o'), (77, '_'), (78, 'f'), (79, 'q'), (80, '4'), (81, '>'), (82, 'l'), (83, ']')])

Use decoderto construct encoder.

In [43]:
#encoder
encoder = {c:i for i, c in decoder.items()}

encoder.items()

dict_items([('H', 0), ('K', 1), ('}', 2), ('C', 3), (';', 4), ('(', 5), ('T', 6), ('I', 7), ('j', 8), ('L', 9), ('u', 10), (')', 11), ('N', 12), ('V', 13), ('F', 14), ('\n', 15), ('r', 16), ('0', 17), ('B', 18), ('D', 19), ('E', 20), ('8', 21), ('g', 22), ('m', 23), ('O', 24), (',', 25), ('h', 26), ('A', 27), ('P', 28), ('J', 29), (':', 30), ('k', 31), ('c', 32), ('`', 33), ('S', 34), ('6', 35), ('2', 36), ('x', 37), ('v', 38), ('U', 39), ('"', 40), ('-', 41), ('i', 42), ('W', 43), ('X', 44), ('1', 45), ('s', 46), ('|', 47), (' ', 48), ('w', 49), ('!', 50), ('R', 51), ('Y', 52), ('p', 53), ('&', 54), ('5', 55), ('Q', 56), ('y', 57), ('n', 58), ('[', 59), ('?', 60), ('9', 61), ('b', 62), ('a', 63), ('z', 64), ('3', 65), ('e', 66), ('.', 67), ('M', 68), ('7', 69), ('G', 70), ('t', 71), ('Z', 72), ('d', 73), ("'", 74), ('<', 75), ('o', 76), ('_', 77), ('f', 78), ('q', 79), ('4', 80), ('>', 81), ('l', 82), (']', 83)])

Using encoder dictionary let's encode the whole text data into corpous of numbers.

In [44]:
encoded_text = np.array([encoder[c] for c in text_data])


In [45]:
encoded_text[:100]

array([15, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48,
       48, 48, 48, 48, 48, 45, 15, 48, 48, 14, 16, 76, 23, 48, 78, 63, 42,
       16, 66, 46, 71, 48, 32, 16, 66, 63, 71, 10, 16, 66, 46, 48, 49, 66,
       48, 73, 66, 46, 42, 16, 66, 48, 42, 58, 32, 16, 66, 63, 46, 66, 25,
       15, 48, 48,  6, 26, 63, 71, 48, 71, 26, 66, 16, 66, 62, 57, 48, 62,
       66, 63, 10, 71, 57, 74, 46, 48, 16, 76, 46, 66, 48, 23, 42])

We just converted whole text data into collection of some numbers.

### Onehot encoding

we need to one-hot encode our data inorder for it to work with the network structure.


In [47]:
def one_hot_encoder(encoded_text, num_uni_chars):
    '''
    encoded_text : batch of encoded text
    
    num_uni_chars = number of unique characters (len(set(text)))
    '''
    
    # METHOD FROM:
    # https://stackoverflow.com/questions/29831489/convert-encoded_textay-of-indices-to-1-hot-encoded-numpy-encoded_textay
      
    # Create a placeholder for zeros.
    one_hot = np.zeros((encoded_text.size, num_uni_chars))
    
    # Convert data type for later use with pytorch (errors if we dont!)
    one_hot = one_hot.astype(np.float32)

    # Using fancy indexing fill in the 1s at the correct index locations
    one_hot[np.arange(one_hot.shape[0]), encoded_text.flatten()] = 1.0
    

    # Reshape it so it matches the batch sahe
    one_hot = one_hot.reshape((*encoded_text.shape, num_uni_chars))
    
    return one_hot

In [51]:
one_hot_encoder(np.array([1,2,0]), 3)

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)

## Batch Generation

We need to create a function that will generate batches of characters along with the next character in the sequence as a label.

In [53]:
len(encoded_text)

5445609

In [63]:
encoded_text[:-1].reshape(2, -1)

array([[15, 48, 48, ..., 66, 48, 23],
       [76, 38, 66, ..., 48, 20, 12]])

In [64]:
def generate_batches(encoded_text, samp_per_batch=10, seq_len=50):
    
    '''
    Generate (using yield) batches for training.
    
    X: Encoded Text of length seq_len
    Y: Encoded Text shifted by one
    
    Example:
    
    X:
    
    [[1 2 3]]
    
    Y:
    
    [[ 2 3 4]]
    
    encoded_text : Complete Encoded Text to make batches from
    batch_size : Number of samples per batch
    seq_len : Length of character sequence
       
    '''
    
    # Total number of characters per batch
    # Example: If samp_per_batch is 2 and seq_len is 50, then 100
    # characters come out per batch.
    char_per_batch = samp_per_batch * seq_len
    
    
    # Number of batches available to make
    # Use int() to roun to nearest integer
    num_batches_avail = int(len(encoded_text)/char_per_batch)
    
    # Cut off end of encoded_text that
    # won't fit evenly into a batch
    encoded_text = encoded_text[:num_batches_avail * char_per_batch]
    
    
    # Reshape text into rows the size of a batch
    encoded_text = encoded_text.reshape((samp_per_batch, -1))
    

    # Go through each row in array.
    for n in range(0, encoded_text.shape[1], seq_len):
        
        # Grab feature characters
        x = encoded_text[:, n:n+seq_len]
        
        # y is the target shifted over by 1
        y = np.zeros_like(x)
       
        #
        try:
            y[:, :-1] = x[:, 1:]
            y[:, -1]  = encoded_text[:, n+seq_len]
            
        # FOR POTENTIAL INDEXING ERROR AT THE END    
        except:
            y[:, :-1] = x[:, 1:]
            y[:, -1] = encoded_text[:, 0]
            
        yield x, y

In [65]:
sample_text = encoded_text[:20]

In [72]:
batch_generator = generate_batches(sample_text, samp_per_batch=2, seq_len=5)

In [71]:
next(batch_generator)

(array([[15, 48, 48, 48, 48],
        [48, 48, 48, 48, 48]]),
 array([[48, 48, 48, 48, 48],
        [48, 48, 48, 48, 48]]))

In [73]:
# Grab first batch
x, y = next(batch_generator)

In [74]:
x

array([[15, 48, 48, 48, 48],
       [48, 48, 48, 48, 48]])

In [75]:
y

array([[48, 48, 48, 48, 48],
       [48, 48, 48, 48, 48]])

We are done with data preparation and also we are good with batch generation for training our model 

## Model Building